# CIFAR 10 dataset - Numpy

In [58]:
%load_ext autotime

The autotime extension is already loaded. To reload it, use:
  %reload_ext autotime
time: 299 µs (started: 2021-04-15 18:36:49 +02:00)


In [59]:
import os
import platform
import pickle
import numpy as np

import torch  

time: 238 µs (started: 2021-04-15 18:36:50 +02:00)


In [60]:
def load_pickle(f):
    version = platform.python_version_tuple()
    if version[0] == '2':
        return  pickle.load(f)
    elif version[0] == '3':
        return  pickle.load(f, encoding='latin1')
    raise ValueError("invalid python version: {}".format(version))

time: 311 µs (started: 2021-04-15 18:36:51 +02:00)


In [61]:
def load_CIFAR_batch(filename):
    """ load single batch of cifar """
    with open(filename, 'rb') as f:
        datadict = load_pickle(f)
        X = datadict['data']
        Y = datadict['labels']
        X = X.reshape(10000,3,32,32)
        Y = np.array(Y)
        return X, Y

time: 300 µs (started: 2021-04-15 18:36:51 +02:00)


In [62]:
def load_CIFAR10(ROOT):
    """ load all of cifar """
    xs = []
    ys = []
    for b in range(1,6):
        f = os.path.join(ROOT, 'data_batch_%d' % (b, ))
        X, Y = load_CIFAR_batch(f)
        xs.append(X)
        ys.append(Y)
    Xtr = np.concatenate(xs)
    Ytr = np.concatenate(ys)
    del X, Y
    Xte, Yte = load_CIFAR_batch(os.path.join(ROOT, 'test_batch'))
    return Xtr, Ytr, Xte, Yte


time: 366 µs (started: 2021-04-15 18:36:52 +02:00)


In [63]:
def get_CIFAR10_data(num_training=49000, num_validation=1000, num_test=10000):
    # Load the raw CIFAR-10 data
    cifar10_dir = 'CIFAR-10-DS/cifar-10-batches-py/'
    X_train, y_train, X_test, y_test = load_CIFAR10(cifar10_dir)

    # Subsample the data
    mask = range(num_training, num_training + num_validation)
    X_val = X_train[mask]
    y_val = y_train[mask]
    mask = range(num_training)
    X_train = X_train[mask]
    y_train = y_train[mask]
    mask = range(num_test)
    X_test = X_test[mask]
    y_test = y_test[mask]

    x_train = X_train.astype('float32')
    x_test = X_test.astype('float32')

    x_train /= 255
    x_test /= 255

    #return X_train, y_train, X_test, y_test
    #return x_train, y_train, X_val, y_val, x_test, y_test 

    return x_train, y_train, x_test, y_test 

time: 406 µs (started: 2021-04-15 18:36:52 +02:00)


In [64]:
# Invoke the above function to get our data.
x_train, y_train, x_test, y_test = get_CIFAR10_data()

print('x_train ', type(x_train), x_train.shape)
print('y_train', y_train.shape)

print('x_test', x_test.shape)
print('y_test', y_test.shape)



x_train  <class 'numpy.ndarray'> (49000, 3, 32, 32)
y_train (49000,)
x_test (10000, 3, 32, 32)
y_test (10000,)
time: 627 ms (started: 2021-04-15 18:36:53 +02:00)


### Create batches

In [65]:
def createBatches(data,batch_size_required):
    batch_size = int(len(data)/batch_size_required)
    res=[]
    print('Data ',len(data))
    for i in range (batch_size):
        batched_data = data[i*batch_size_required:i*batch_size_required+batch_size_required]
        res.append(batched_data)
        #print('Check here', len(batched_data))
    res = np.asarray(res)
    print('Res ',len(res))
    return res

time: 352 µs (started: 2021-04-15 18:36:57 +02:00)


In [66]:
batch_size = 64
x_train = createBatches(x_train,64)
y_train = createBatches(y_train,64)

x_test = createBatches(x_test,64)
y_test = createBatches(y_test,64)

Data  49000
Res  765
Data  49000
Res  765
Data  10000
Res  156
Data  10000
Res  156
time: 322 ms (started: 2021-04-15 18:37:00 +02:00)


## Convert npArray to tensor

In [67]:
x_train_tensor = torch.as_tensor(x_train)
y_train_tensor = torch.as_tensor(y_train)

x_test_tensor = torch.as_tensor(x_test)
y_test_tensor = torch.as_tensor(y_test)

print('X tensor ', type(x_test_tensor))

X tensor  <class 'torch.Tensor'>
time: 2.62 ms (started: 2021-04-15 18:37:38 +02:00)


In [29]:
#transform = transforms.Compose([transforms.ToTensor()])

#trainset = torchvision.datasets.CIFAR10(root='./data', train=True,download=True, transform=transform)


#trainloader = torch.utils.data.DataLoader(trainset, batch_size=bth_size,shuffle=True, num_workers=2)

#testset = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=transform)

#testloader = torch.utils.data.DataLoader(testset, batch_size=bth_size, shuffle=False, num_workers=2)

### Preview CIFAR-10 dataset images

In [68]:
import matplotlib.pyplot as plt
import numpy as np

# functions to show an image


def imshow(img):
    #img = img / 2 + 0.5     # unnormalize
    npimg = img.numpy()
    print(img.shape)
    plt.imshow(np.transpose(img))
    plt.show()


# get some random training images
#dataiter = iter(trainloader)
#print(dataiter)
#images, labels = dataiter.next()
images = x_train_tensor
labels = y_train_tensor

# show images
imshow(images[50])
# print labels
#print(' '.join('%5s' % classes[labels[j]] for j in range(32)))

torch.Size([64, 3, 32, 32])


TypeError: Invalid shape (32, 32, 3, 64) for image data

### Setting up the layers of Convolutional neural network

In [69]:
in_size = 3                 # number of channel in the input image
 
hid1_size = 64              # no of output channel from first CNN layer

hid2_size = 128             # no of output channel from second CNN layer

hid3_size = 256             # no of output channel from third CNN layer

hid4_size = 512             # no of output channel from forth CNN layer

classes = ('plane', 'car', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

out_size = len(classes)     # no of categories in the dataset

k_conv_size = 3             # 3x3 convolutional kernel

conv_stride = 1             # conv stride 1

conv_pad = 1                # conv padding 1

maxpool_kernel = 2          # maxpool layer kernel size 2 x 2

maxpool_stride = 2          # maxpool layer stride 2

time: 411 µs (started: 2021-04-15 18:38:03 +02:00)


In [70]:
import torch.nn as nn

class VGG_11(nn.Module):
    
    def __init__(self):
        super(VGG_11,self).__init__()
        
        self.convLayer = nn.Sequential(
            nn.Conv2d(in_size, hid1_size, k_conv_size, stride=conv_stride, padding=conv_pad),    # conv layer
            nn.BatchNorm2d(hid1_size),
            #nn.LocalResponseNorm(64),
            nn.ReLU(),                              # Activation layer
            nn.MaxPool2d(kernel_size=maxpool_kernel,stride=maxpool_stride),
            
            nn.Conv2d(hid1_size,hid2_size,k_conv_size, stride=conv_stride, padding=conv_pad),
            nn.BatchNorm2d(hid2_size),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=maxpool_kernel,stride=maxpool_stride),
            
            nn.Conv2d(hid2_size,hid3_size,k_conv_size, stride=conv_stride, padding=conv_pad),
            nn.BatchNorm2d(hid3_size),
            nn.ReLU(),
            
            nn.Conv2d(hid3_size,hid3_size,k_conv_size, stride=conv_stride, padding=conv_pad),
            nn.BatchNorm2d(hid3_size),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=maxpool_kernel,stride=maxpool_stride),
            
            nn.Conv2d(hid3_size,hid4_size,k_conv_size, stride=conv_stride, padding=conv_pad),
            nn.BatchNorm2d(hid4_size),
            nn.ReLU(),
            
            nn.Conv2d(hid4_size,hid4_size,k_conv_size, stride=conv_stride, padding=conv_pad),
            nn.BatchNorm2d(hid4_size),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=maxpool_kernel,stride=maxpool_stride),
            
            nn.Conv2d(hid4_size,hid4_size,k_conv_size, stride=conv_stride, padding=conv_pad),
            nn.BatchNorm2d(hid4_size),
            nn.ReLU(),
            
            nn.Conv2d(hid4_size,hid4_size,k_conv_size, stride=conv_stride, padding=conv_pad),
            nn.BatchNorm2d(hid4_size),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=maxpool_kernel,stride=maxpool_stride), 
            
            #nn.Flatten(0,-1),
            
            #nn.Linear(512*bth_size, out_size)
            
        )
        
     
        self.fullyConnLayer = nn.Sequential(
            nn.Linear(512, out_size),
            #nn.Softmax(dim=1)
        )
            
        
    def forward(self,x):
            out = self.convLayer(x)

            out = out.reshape(out.size(0), -1)
            
            out = self.fullyConnLayer(out)
            
            return out 
        
vgg_11 = VGG_11()

time: 43.2 ms (started: 2021-04-15 18:38:03 +02:00)


## Define a Loss function and optimizer

In [71]:
import torch.optim as optim

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adadelta(vgg_11.parameters())

time: 540 µs (started: 2021-04-15 18:38:05 +02:00)


## Train the network

In [72]:
for epoch in range(1):  # loop over the dataset multiple times

    running_loss = 0.0
    for i, data in enumerate(x_train_tensor, 0):
        #inputs, labels = data
        
        #images = data
        #print(data.shape)
        inputs = data
        labels = y_train_tensor[i]
        optimizer.zero_grad()
        outputs = vgg_11(inputs)
        
        #print(type(outputs))
        #print(type(labels))
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        
        # print statistics        
        if i % 200 == 199:    # print every 200 mini-batches
            print('[%d, %5d] loss: %.3f' %
                  (epoch + 1, i + 1, running_loss / 200))
            running_loss = 0.0

print('Finished Training')

[1,   200] loss: 2.287
[1,   400] loss: 1.696
[1,   600] loss: 1.451
Finished Training
time: 12min 55s (started: 2021-04-15 18:38:07 +02:00)


## Test the network on the test data

In [ ]:
dataiter = iter(testloader)
images, labels = dataiter.next()

# print images
imshow(torchvision.utils.make_grid(images))
print('GroundTruth: ', ' '.join('%5s' % classes[labels[j]] for j in range(32)))

In [ ]:
outputs = vgg_11(images)

In [ ]:
_, predicted = torch.max(outputs, 1)

print('Predicted: ', ' '.join('%5s' % classes[predicted[j]]
                              for j in range(32)))

### Accuracy on whole test-dataset

In [73]:
correct = 0
total = 0
with torch.no_grad():
    for i,data in enumerate(x_test_tensor, 0):
        #images, labels = data
        #print('test ', len(data))
        images = data
        labels = y_test_tensor[i]

        outputs = vgg_11(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print('Accuracy of the network on the 10000 test images: %d %%' % (
    100 * correct / total))

Accuracy of the network on the 10000 test images: 56 %
time: 24.1 s (started: 2021-04-15 18:51:44 +02:00)


### Which classes perform well and which classes did not?

In [75]:
class_correct = list(0. for i in range(10))
class_total = list(0. for i in range(10))
with torch.no_grad():
    for i, data in enumerate(x_test_tensor, 0):
        #images, labels = data
        images = data
        labels = y_test_tensor[i]

        outputs = vgg_11(images)
        _, predicted = torch.max(outputs, 1)
        c = (predicted == labels).squeeze()
        for i in range(4):
            label = labels[i]
            class_correct[label] += c[i].item()
            class_total[label] += 1


for i in range(10):
    print('Accuracy of %5s : %2d %%' % (
        classes[i], 100 * class_correct[i] / class_total[i]))

Accuracy of plane : 65 %
Accuracy of   car : 78 %
Accuracy of  bird : 15 %
Accuracy of   cat : 54 %
Accuracy of  deer : 27 %
Accuracy of   dog : 20 %
Accuracy of  frog : 73 %
Accuracy of horse : 46 %
Accuracy of  ship : 80 %
Accuracy of truck : 60 %
time: 23.7 s (started: 2021-04-15 18:52:43 +02:00)


# END

# Testing

In [39]:
arrays = [np.random.randn(3, 32,32) for _ in range(128)]

In [77]:
test = np.stack(arrays, axis=0)

In [90]:
test.shape

(128, 3, 32, 32)

In [ ]:
#res = np.stack(test,axis=)

In [53]:
res = np.array(np.split(test, 64))

In [54]:
res.shape

(64, 2, 3, 32, 32)

In [59]:
batch_size = 64
no_of_batches = abs(len(test)/batch_size)

In [60]:
no_of_batches

2.0

In [82]:
def createBatches(data,batch_size_required):
    batch_size = int(len(data)/batch_size_required)
    res=[]
    print(len(data))
    for i in range (batch_size):
        batched_data = data[i*batch_size_required:i*batch_size_required+batch_size_required]
        res.append(batched_data)
        print('Check here', len(batched_data))
    res = np.asarray(res)
    return res

In [91]:
test.shape

(128, 3, 32, 32)

In [92]:
b_d = createBatches(test,64)

128
Check here 64
Check here 64


In [93]:
len(b_d)

2

In [94]:
resttt = np.asarray(b_d)

In [96]:
resttt[0].shape

(64, 3, 32, 32)